In [ ]:
#Instalação BERT
!pip install -U sentence-transformers

In [2]:
#from sentence_transformers import SentenceTransformer
import csv
import pandas as pd
from pathlib import Path
import numpy as np
from numpy import savetxt
import random
from random import sample

In [7]:
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/DBERT/Fold8/')
files_in_basepath = basepath.iterdir()
count =0
for item in files_in_basepath:
  if item.is_file():
    count+=1
count

183

In [ ]:
def embeddingDistilBERTML(txts,nome_base):

  print('na base: ' + nome_base)

  model = SentenceTransformer('distiluse-base-multilingual-cased')
  
  sentences=txts.replace(['\t','\n','\r'], [' ',' ',' '], regex=True)

  sentence_embeddings = model.encode(sentences)

  arquivo = open('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT/' + nome_base + '_DistilBERTML.csv','w')
  for line in np.array(sentence_embeddings):
    arquivo.write(",".join(str(x) for x in line)+'\n')
  arquivo.close()

In [ ]:
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/News/')
files_in_basepath = basepath.iterdir()
bases = {}
for item in files_in_basepath:
  if item.is_file():
    df = pd.read_csv('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/News/' + item.name)
    embeddingDistilBERTML(df['Title'], item.name.replace('.csv',''))

In [ ]:
dic_DistilBERT = {}
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    arq = open('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT/' + item.name, 'r')
    mat_embedding = list()
    reader = csv.reader(arq)
    for doc in reader:
      mat_embedding.append([float(i) for i in doc])
    dic_DistilBERT[item.name.replace('.csv','')] = np.array(mat_embedding)

In [ ]:
seed = 10 # sementes escolhidas: 1,2,3,4,5,6,7,8,9,10

bases_len = len(dic_DistilBERT.keys())

for base in dic_DistilBERT.keys():
  data = dic_DistilBERT[base]
  data_total = data
  data_len = len(data)
  train_len = int(data_len/3)
  test_len = data_len - train_len
  outlier_cada_base = int(test_len/(bases_len-1))
  for b in dic_DistilBERT.keys():
    if b != base:
      data_b_len = len(dic_DistilBERT[b])
      train_b_len = int(data_b_len/3)
      random.seed(seed)
      sorteados = sample(range(train_b_len, data_b_len), outlier_cada_base)
      for i in sorteados:
        data_outlier = np.array(dic_DistilBERT[b][i]).reshape(1,512)
        data_total = np.concatenate([data_total,data_outlier])
      
  name = '/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT-WITH-OTHER-CLASSES/Fold'+ str(seed) + '/' + base + '_' +str(train_len)+ '_' +str(data_len)+'.csv'
  savetxt(name, data_total, delimiter=',')

In [ ]:
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT-WITH-OTHER-CLASSES/Fold10/')
files_in_basepath = basepath.iterdir()
count =0
for item in files_in_basepath:
  if item.is_file():
    count+=1
count

183